<div class="alert alert-block alert-info">
<b>What is done here:</b> Use the PT_Get_Text.py to apply it to other texts.</div>

In [ ]:
import PT_Get_Text
import pandas as pd
import pickle
import os

In [1]:
def limpa_texto(texto):
    import re
    texto = texto.replace('\\n','')
    
    texto = re.sub(r"([0-9]*[\.,]{0,2}[0-9])|(\d? \d)", '', texto)

    lista_de_vogais = ['A','E','I','O','U','a','e','i','o',
                   'u','Á','Ã','À','Ä','Â','á','ã','à',
                   'ä','â','É','È','Ë','Ê','é','è','ë',
                   'ê','Í','Ì','Ï','Î','í','ì','ï','î',
                   'Ó','Õ','Ò','Ö','Ô','ó','õ','ò','ö',
                   'ô','Ú','Ù','Ü','Û','ú','ù','ü','û']

    lista_de_consoantes = ['B','C','Ç','D','F','G','H','J','K',
                           'L','M','N','Ñ','P','Q','R','S','T',
                           'V','W','X','Y','Z','b','c','ç','d',
                           'f','g','h','j','k','l','m','n','ñ',
                           'p','q','r','s','t','v','w','x','y','z']

    lista_de_pontuacao = ['.',':','.','…','!','?',',',
                          '-','–',
                          '(',')',
                          '“','”','"',"'","/",
                          ' ']
    
    return "". join([char for char in texto if char in lista_de_vogais or char in lista_de_consoantes or char in lista_de_pontuacao]).replace(' .,','').replace(' .,','')

def Predict_Year(text):
    '''
    Recebe um texto (string) e devolve:
    [0] -> Previsão do ano de escolaridade. [1-12]
    [1] -> Previsão do difículdade baseada no ano de escolaridade [1-8]
    '''
#     text = limpa_texto(text)

    list_cols = ['Words per sentence','Syllables per Word','Poly_Syl per Word','Verbs per Word','Noun per Word',
                 'Adj per Word','Simp_Word_DC per Word','Simp_Word_1 per Word','Uniques_Per_Word','HH_Index',
                 'count_SVO_found per word','count_SVO_found_in_a_row per word','count_SVO_elements_in_a_row per word']

    stats = PT_Get_Text.Get_All_Text_Info(text)
    map_columns = {n:i for n, i in zip(range(len(list_cols)), list_cols)}
    DF = pd.DataFrame(stats).T.rename(map_columns, axis = 1)
    
    xg_reg_ai = pickle.load(open("xg_reg_ai.dat", "rb"))
    xg_reg_ne = pickle.load(open("xg_reg_ne.dat", "rb"))
    ano_pred = xg_reg_ai.predict(DF)
    ne_pred = xg_reg_ne.predict(DF)
    
    return (ano_pred[0], ne_pred[0])

def load_text(path, encoding = 'UTF-8'):
    f = open(path, "r", encoding = encoding)
    raw_text = f.read()    
    raw_text = raw_text.replace('\n',' ')
    raw_text = raw_text.replace('    ',' ')
    return raw_text

def getListOfFiles(dirName):
    import os
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

In [2]:
#### Example calling out the function:
fifth_grader_text = '''
Resolvi chegar-me à janela e ver.
Um vento cortante e desagradável movia os tristes raminhos, já mortos, de uma árvore deitada contra o pavimento esburacado da rua. Espectros enormes e claros no escuro da noite, erguiam-se a todo o longo da rua -eram o que restava de casas que tinham vivido, alegres, em tempos mais felizes. Um velho tanque, amolgado e dilacerado dava uma nota ainda mais sombria a este cenário já de si tão aterrador. 
De quando em quando, sentia-se o rolar sinistro de uma pedra, batendo contra o chão. Um ruído triste, vago como um murmúrio!...
No pouco que restava do passeio, dando pontapés nos escombros que se lhe atravessavam no caminho, um homem fardado prosseguia a sua ronda. Os seus passos soavam, incansáveis, monótonos, irritantes. Numa casa ainda habitável, mesmo defronte, um grupo de pessoas , a avaliar pelas vozes que, de vez em quando se faziam ouvir, tentavam sintonizar um aparelho de rádio, querendo, talvez, saber quando acabaria “aquilo”.
Ouviam-se, longe, notas altas e desafinadas.
Era um velho piano…
Passou-se uma hora lenta, muito lenta. E tudo continuava calmo. Mas, de repente, no meio da calma da noite fria, começou a ouvir-se um silvo penetrante e agudíssimo, lamentoso, como que a anunciar a morte.
A rua pareceu viver. Foi atravessada em todas as direções por pessoas com o terror estampado no rosto, que procuravam refúgios nas ruínas e entrada nos subterrâneos e esconderijos.
O piano calou-se abruptamente.
Gritaram-me de dentro:
- Depressa, para “baixo”!
Em baixo era o nosso refúgio, desde que aquela sereia irritante nos chamava para enfrentar a morte.
Deixei o sítio onde estava e precipitei-me para as escadas, embora pensando de que nada valeria. Fechei os olhos e cerrei os punhos com força, à espera de morrer. Mesmo sem querer só podia pensar se iria doer muito. Mas, se doesse, também não fazia mal- era a última vez que doía!
Durou dez minutos. Dez minutos dos quais eu saí mais velha dez anos. Dez minutos que pareceram dez séculos. Finalmente, a maldita serei parou. Estávamos salvos, salvos! Ainda estava viva…
Sim, mas quanto tempo mais?
Saí à rua.
 Num dos ramos da árvore caída, esvoaçava um bocadinho de pano azul… da farda azul do homem que fazia a ronda.
 Pessoas removiam corpos sem uma lágrima, como rosto endurecido, resignado.
- Não morremos desta! – disseram uns.
-Pois não, é para a próxima! -  responderam, com voz rouca e raivosa, outros.
Era a guerra!
'''          
Predict_Year(limpa_texto(fifth_grader_text))

###### Looking at some newspaper articles

In [14]:
# These files were in sub-folders one for each newspaper.
# I personally used a paid subscription to Factiva... not uploading here the files, copyrighted.
file_names = []
scores = []

my_list_of_files = getListOfFiles('C:\\Users\\MYUSER\\[...]') #### Wrute the parent directory here.

for i in range(len(my_list_of_files)):
    file_names.append(my_list_of_files[i].replace('C:\\Users\\MYUSER\\[...]',""))
    scores.append(Predict_Year(limpa_texto(load_text(my_list_of_files[i]))))

JornaisDF = pd.DataFrame(list(zip([file.split('\\')[0] for file in file_names],
                                  [file.split('\\')[1] for file in file_names],
                                  [score[0] for score in scores],
                                  [score[1] for score in scores])))

JornaisDF = JornaisDF.rename({0:'Jornal',1:'Artigo',2:'Idade',3:'NE'}, axis = 1)

{'tokenizer': True, 'pos_tagger': True, 'lemmatizer': True, 'entity_recognition': False, 'np_chunking': False, 'pre_load': True, 'string_or_array': True}


In [25]:
JornaisDF.describe()

,Idade,NE
count,54.000000,54.000000
mean,4.586684,4.839627
std,1.127124,0.737889
min,2.731640,2.337756
25%,3.831310,4.422168
50%,4.436515,4.904546
75%,5.200746,5.403459
max,8.621509,5.854350


In [16]:
JornaisDF.groupby('Jornal').mean()

,Idade,NE
Jornal,,
Correio da Manha,4.735869,5.259427
Jornal de Negócios,4.021949,4.258791
Observador,4.568733,4.934166
Publico,5.075145,4.824700


In [1]:
##### Random text from Banco de Portugal's Boletim Económico Publication.
Boletim_Economico = '''
A recuperação da atividade mundial ocorre ao longo do horizonte de projeção. As hipóteses para o enquadramento internacional têm subjacente um crescimento económico mundial de 6,2%
em 2021 e um abrandamento para 4,0% e 3,5% em 2022 e 2023 (Quadro I.2.1). O crescimento foi revisto em alta especialmente em 2021 devido ao crescimento mais forte do que o antecipado no
final de 2020, ao acordo de comércio entre o Reino Unido e a União Europeia e ao pacote de estímulo orçamental dos Estados Unidos da América anunciado em dezembro. Para a área do euro,
as projeções do Banco Central Europeu (BCE) apresentam um crescimento económico de 4,0% em
2021, 4,1% em 2022 e 2,1% em 2023, praticamente inalterado face ao exercício de dezembro. No
final de 2020 e início de 2021, a atividade na área do euro tem sido afetada pelo recrudescimento
da pandemia e das medidas de confinamento, estimando-se um impacto mais reduzido do que
o observado no primeiro trimestre do ano passado.
O padrão de crescimento global é diferenciado entre regiões e setores de atividade. As economias de mercado emergentes, em particular a China, têm registado um crescimento mais dinâmico
do que as economias avançadas. Adicionalmente, a recuperação do comércio mundial deverá ser
mais atenuada nos serviços, em particular nas viagens e turismo. Por seu lado, as trocas internacionais de bens têm revelado maior resiliência. Neste contexto, a procura externa deverá recuperar
menos do que o comércio mundial, com um crescimento de 7,4% em 2021, desacelerando para 3,5% em 2023.
Os preços das matérias-primas aumentam, refletindo sobretudo a recuperação da atividade.
As hipóteses atuais incluem um aumento do preço médio do petróleo face ao anterior exercício
de projeção, para 59,3 dólares por barril em 2021, descendo gradualmente para 53,7 dólares em
2023. As hipóteses incluem uma apreciação do euro de 1,8% em 2021, refletindo a apreciação
dos últimos meses face ao dólar.
As condições monetárias e financeiras deverão manter-se favoráveis ao longo do horizonte
de projeção, sustentadas por políticas monetárias acomodatícias. Na área do euro, assumem-se taxas de juro de curto e longo prazo estáveis e próximas dos níveis atuais ao longo de todo
o horizonte de projeção. A taxa de juro implícita da dívida pública portuguesa apresenta um perfil
descendente, de 2,0% em 2021 até 1,8% em 2023.
'''
Predict_Year(limpa_texto(Boletim_Economico))